#M5 Forecasting

Kaggle's competition M5 Forecasting - Accuracy dataset consist of Walmart's sales dataset across their shops in 3 states of America. In this notebook we will try to predict sales for 28 days.

##Model

Facebook's Prophet will be model of our choice.
- easy to use without expert knowledge on statistics or time series forecasting
- allows to use multiple seasonabilities 
- easy to input holiday effects

##Objectives
- inspect and analyse provided datasets
- extract holiday from calendar and use it within our model
- train our model and predict future 28 days of sales for each item

##Imports

In [ ]:
import pandas as pd
import pickle
from tqdm import tqdm

from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation , performance_metrics
from fbprophet.plot import plot_cross_validation_metric

import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.float_format = '{:20,.2f}'.format 

##Variables

By assigning dtypes we can reduce RAM usage for each dataframe.
- `sell_prices` was reduced from 0.94GB to 0.86GB
- `sales_train_val` was reduced from 0.45GB to 0.07GB
- `calendar` dataframe size is only 0.09MB , so we left it untouched



In [ ]:

calendar=pd.read_csv('/Users/hmelino/Desktop/Coding/m5-forecasting-accuracy/calendar.csv')
sales_train_val=pd.read_csv('/Users/hmelino/Desktop/Coding/m5-forecasting-accuracy/sales_train_validation.csv',dtype={f'd_{v}':'int8' for v in range(1,1914)})
#sample_sub=pd.read_csv('/content/drive/My Drive/Reports/m5-forecasting-accuracy/sample_submission.csv')[:30490]
sell_prices=pd.read_csv('/Users/hmelino/Desktop/Coding/m5-forecasting-accuracy/sell_prices.csv',dtype={'wm_yr_wk':'int16','sell_price':'float16'})

##Exploring data

In [ ]:
sales_train_val

sales_train_val is the main dataset we will use. Each row contains `item_id` , deparment, category , `store_id` , `state_id` and over 5 years of sales ( 1913 days ). 

##Holiday

Holiday feature helps us capture seasonability of dataset. With use of `calendar` dataset , we will :

- translate '**wm_yr_wk**' value `11101` into `2011-01-29` for our **sell_price** dataset

- create **holiday** dataset for our Prophet model

In [ ]:
calendar.head(3)

Using dictionary comprehension we will extract and order all events with coresponding dates.

In [ ]:
holiday_dict = {holidayName:[date for date in calendar.loc[calendar['event_name_1']==holidayName]['date'].values] for holidayName in calendar['event_name_1'].unique()}

In [ ]:
holiday_dict

In [ ]:
holiday_list=[pd.DataFrame({'holiday':hol_name,'ds':pd.to_datetime(dates)}) for hol_name,dates in holiday_dict.items()]

In [ ]:
holiday_list[1]

All converted dataframes will be now atatched to main holiday dataframe.

In [ ]:
%%time
holiday = pd.DataFrame(columns=['holiday','ds'])
for d in holiday_list:
  holiday = pd.concat([holiday,d])

In [ ]:
#release RAM and remove unececssary variables
del holiday_list
del holiday_dict

In [ ]:
holiday

##X_train transfomation

`sales_train_val` contains all training values for our model.

One of the best ways to start data transforming is to remove unecessary columns.

In [ ]:
%%time
df=sales_train_val.drop(['item_id','dept_id','cat_id','store_id','state_id'],axis=1).set_index(sales_train_val['id'].str[:-11]).T[1:].set_index(calendar['date'][:1913])

cat_list = list(df.columns)
future = pd.DataFrame({'ds':pd.date_range(start='2016-04-25',end='2016-05-23',freq='d')})
column = 'HOBBIES_1_001_CA_1'

#free up memory and remove unecessary variable
del sales_train_val

In [ ]:
df.to_csv('df.csv')

By default, Facebook Prophet will include training data in predictions.

However with such a big dataset, every second of computing time is precious. To speed up such a time consuming task we will specify 28 days we want to predict.

In [ ]:
future = pd.DataFrame({'ds':pd.date_range(start='2016-04-25',end='2016-05-26')})

##Fit & Predict

By specifying parameters, our model will be more accurate and faster.

- `daily_seasonality=False`,`weekly_seasonality=False`,`yearly_seasonality=True` - specify seasonability
- `uncertainity_samples=0` as we dont need values for yhat_lower and yhat_upper
- `holidays` - seasonability, ability to catch extra high and extra low selling days

In [ ]:
%%time
m = Prophet(daily_seasonality=False,weekly_seasonality=False,yearly_seasonality=True,uncertainty_samples=0,holidays=holiday)
data = pd.DataFrame({'ds':df[column].index,'y':df[column].values})
m.fit(data)
predicted_values = m.predict(future)['yhat'].values

We have trained and predicted future 28 days of sales for column `HOBBIES_1_001_CA_1`.
But how good are our predicitons ? Cross validation is a good tool to find out accuracy of our model.

Luckily , Prophet comes already with cross validation tool.

In [ ]:
df_cross_val = cross_validation(m,initial='1095 days',period='365 days',horizon='20 days' )

In [ ]:
df_cross_val.head()

In [ ]:
performance_metrics(df_cross_val)

In [ ]:
plot_cross_validation_metric(df_cross_val,metric='mae')

Our model is doing pretty well, as `MAE` - mean absolute error is withing appropriate range.

#Fit & Predict 30490 row


In [ ]:
%%time
def run_prophet(column):
  m = Prophet(daily_seasonality=False,weekly_seasonality=False,yearly_seasonality=True,uncertainty_samples=0,holidays=holiday)
  data = pd.DataFrame({'ds':df[column].index,'y':df[column].values})
  m.fit(data)
  return m.predict(future)['yhat'].values

run_prophet(column)

Now we have our function that will predict 28 days of sales for given column/item. 

Cell below will use this function to predict 30490 columns/items. On Google's Cloud with 16 cpu's it took just under an hour to complete. 

To avoid running this cell every single time we run this notebook , I have saved output into pickle file , and commented out whole cell, to prevent it from running.

In [ ]:
'''
from multiprocessing import Pool,cpu_count
from tqdm import tqdm
p = Pool(cpu_count())
predictions = list(tqdm(p.imap(run_prophet,cat_list),total=len(cat_list)))
p.close()
p.join()

import pickle
with open('new_prediction.pickle','wb') as pickle_file:
  pickle.dump(predictions,pickle_file)
'''

\[OUT]
100%|██████████| 30490/30490 \[58:32<00:00,  8.68it/s] 

Lets load our saved predictions.

In [ ]:
predictions=pickle.load(open('/Users/hmelino/Desktop/Coding/m5-forecasting-accuracy/new_prediction.pickle','rb'))

To double check that we have all predicted values, we will use len() function.

In [ ]:
len(predictions)

Everything is going according to plan.

#Organise forecast data

As we know , we are still dealing with a large size of data and while designing the fastest function, I have learned a lot about how Pandas and Python work with data. 

##My first attempt:

[IN]
```
%%time
c_names = [f'F{n}' for n in range(1,29)]
final_df = pd.DataFrame(columns=['id']+c_names)
for i in range(len(future_sales)):
  new_row_df = pd.DataFrame(future_sales[i],index=c_names).T
  new_row_df['id']=df.columns[i]
  final_df=pd.concat([final_df,new_row_df])
```
[OUT]


```
CPU times: user 1min 50s, sys: 1.06 s, total: 1min 51s
Wall time: 1min 52s
```

... and improved version :)


In [ ]:
#%%time
c_names = [f'F{n}' for n in range(1,29)]

forecast = pd.concat([pd.DataFrame(predictions[i][:28],index=c_names).T for i in tqdm(range(len(predictions)))])

forecast['id']=[v for v in df.columns]

As we can see , there is always a room for improvement , our new function is 7x faster.

We have applied these improvements:
- using list comprehension

- concatenating all dataframes together

In [ ]:
forecast

Now we have organised forecast data, we can attach them to main dataset and visualise our predictions.

In [ ]:
plt.figure(figsize=[25,10])
states = {'CA':'California','WI':'Wisconsin','TX':'Texas'}

for c in range(1,len(states.keys())+1):
  plt.subplot(3,1,c) #position of plot
  plt.xlim(1050,1950) #cut first 1050 days as there were no sales
  state_code = list(states.keys())[c-1] #get state code from states dict

  plt.title(states[state_code]) #set title of plot to state name
  plot_df = pd.DataFrame({'y': list(df[f'HOBBIES_1_354_{state_code}_1'].values.astype('float')) + list(forecast.loc[forecast['id']==f'HOBBIES_1_008_{state_code}_1'][[c for c in forecast.columns if 'F' in c]].values[0])  })
  sns.lineplot(data=plot_df,x=plot_df.index,y='y')
  plt.gca().set(xticklabels = [pd.date_range(start = '2011-01-29', end= '2016-05-26')[int(1941/11*v)].strftime('%Y-%m-%d') for v in range(10)])

  plt.axvspan(1912,1940,alpha=0.3,label='prediction',color='red')
  plt.tight_layout(pad=3.0)
plt.show()

We have sucesfully predicted 28 days of sales for each item in our dataset. 

Analysis comes in separate notebook very soon ...  

Thank you for your attention :)